In [167]:
import tensorflow as tf

!pip install librosa

In [168]:
import numpy as np

In [169]:
file="data/mini_speech_commands/down/ffd2ba2f_nohash_4.wav"
record="enregistrement_audio.wav"
def process_audio(record):
     audio_bin=tf.io.read_file(record)
     audio,sample_rate=tf.audio.decode_wav(audio_bin)
     waveform=tf.squeeze(audio)
     waveform=waveform[:16000]
     zero_padding = tf.zeros(
      [16000] - tf.shape(waveform),
      dtype=tf.float32)
     waveform = tf.cast(waveform, dtype=tf.float32)
     
     spectrogram=tf.signal.stft(signals=waveform,frame_length=255,frame_step=128)
     spectrogram = tf.abs(spectrogram)
     spectrogram=spectrogram[..., tf.newaxis]
     spectrogram=tf.expand_dims(spectrogram,axis=0)
     return spectrogram


spectrogram=process_audio(file)

spectrogram.shape

TensorShape([1, 124, 129, 1])

In [170]:
model_path = "voice3.h5"
# Charger le modèle
model = tf.keras.models.load_model(model_path)
model.input_shape

(None, 124, 129, 1)

In [171]:
def make_prediction(audio_file):
     audio_data=process_audio(audio_file)
     prediction = model.predict(audio_data)
     # Trouver l'indice de la probabilité maximale
     predicted_index = np.argmax(prediction, axis=1)
     predicted_index=predicted_index[0]
     # Trouver la classe prédite en utilisant l'indice
     predicted_class = labels[predicted_index]
     predicted_class
     return predicted_class

In [172]:
pred=make_prediction(file)
pred

1/1 [==============================] - 0s 214ms/step


'go'

In [173]:
pred[0,1]

labels = ['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes']

# Trouver l'indice de la probabilité maximale
predicted_index = np.argmax(pred, axis=1)
predicted_index=predicted_index[0]
# Trouver la classe prédite en utilisant l'indice
predicted_class = labels[predicted_index]
predicted_class

TypeError: string indices must be integers, not 'tuple'

In [ ]:
!pip install pyaudio

  Using cached PyAudio-0.2.13-cp311-cp311-win_amd64.whl (164 kB)


In [ ]:

import numpy as np
from scipy.io import wavfile
import tensorflow as tf
from tensorflow import keras
import pyaudio
import wave
import os
import datetime

def record_audio(duration=5, sample_rate=44100, channels=1):
    format = pyaudio.paInt16
    audio = pyaudio.PyAudio()
    stream = audio.open(
        format=format,
        channels=channels,
        rate=sample_rate,
        input=True,
        frames_per_buffer=1024,
    )

    print("Enregistrement audio en cours...")
    frames = []
    for i in range(0, int(sample_rate / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)

    print("Enregistrement audio terminé.")

    # Fermeture du flux audio
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Création du nom de fichier avec l'horodatage
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_file = os.path.join("recording", f"audio_{timestamp}.wav")

    # Sauvegarde de l'audio en tant que fichier WAV
    wave_file = wave.open(output_file, "wb")
    wave_file.setnchannels(channels)
    wave_file.setsampwidth(audio.get_sample_size(format))
    wave_file.setframerate(sample_rate)
    wave_file.writeframes(b"".join(frames))
    wave_file.close()

    print("Audio enregistré avec succès :", output_file)
    return output_file


In [185]:
!pip install Pillow

In [196]:
import pyaudio
import wave

def play_audio(file_path):
    chunk = 1024  # Nombre d'échantillons lus à chaque fois
    
    # Ouvrir le fichier audio
    wf = wave.open(file_path, 'rb')

    # Initialiser PyAudio
    p = pyaudio.PyAudio()

    # Ouvrir le flux de lecture
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # Lire les données du fichier audio par morceaux et les envoyer au flux de sortie
    data = wf.readframes(chunk)
    while data:
        stream.write(data)
        data = wf.readframes(chunk)

    # Arrêter le flux de sortie et fermer le fichier audio
    stream.stop_stream()
    stream.close()
    p.terminate()

    print("Lecture terminée.")

# Utilisation de la fonction play()
fichier_audio = "recording/audio_2023-07-05_21-22-59.wav"
play_audio(fichier_audio)


Lecture terminée.


In [198]:
import tkinter as tk
from PIL import ImageTk, Image

# Création de la fenêtre principale
window = tk.Tk()
window.title("Logiciel de Reconnaissance Vocal")
window.geometry("600x600")

recording_audio=''
def button_click():  
    label.config(text="Start recording...")
    global recording_audio
    recording_audio=record_audio(duration=5)
    label.config(text="succes recording")
    print(recording_audio)
    

def play():  
     label.config(text="stat playing")  
     play_audio(recording_audio)
     
     
def predicted():
    pred=make_prediction(recording_audio)
    label_result.config(text="vous avez dit:"+pred)






button = tk.Button(window, 
                    text="enregistrer audio",
                    command=button_click,                  
                    padx=10, pady=10)
buton_play=tk.Button(window,
                     text='Play',
                     command=play,
                     padx=10, pady=10)
button_predict=tk.Button(window,
                        text='Pred',  
                        command=predicted,                    
                        padx=10, pady=10)
label_result=tk.Label(window,
                     text='resultat de prediction:',
                     padx=20, pady=20)

label = tk.Label(window, text="Bienvenue sur VoiceR",padx=30,pady=30)

label.grid(row=0,column=0,)
label_result.grid(row=2)

buton_play.grid(row=6, column=4)
button.grid(row=6, column=3)
button_predict.grid(row=6,column=5)

window.grid_rowconfigure(0, minsize=50)
window.grid_columnconfigure(0, minsize=50)


# Lancement de la boucle principale
window.mainloop()



Enregistrement audio en cours...
Enregistrement audio terminé.
Audio enregistré avec succès : recording\audio_2023-07-06_10-04-16.wav
recording\audio_2023-07-06_10-04-16.wav
Lecture terminée.
Lecture terminée.
Enregistrement audio en cours...
Enregistrement audio terminé.
Audio enregistré avec succès : recording\audio_2023-07-06_10-04-43.wav
recording\audio_2023-07-06_10-04-43.wav
Lecture terminée.
Enregistrement audio en cours...
Enregistrement audio terminé.
Audio enregistré avec succès : recording\audio_2023-07-06_10-04-57.wav
recording\audio_2023-07-06_10-04-57.wav
Enregistrement audio en cours...
Enregistrement audio terminé.
Audio enregistré avec succès : recording\audio_2023-07-06_10-05-02.wav
recording\audio_2023-07-06_10-05-02.wav
Enregistrement audio en cours...
Enregistrement audio terminé.
Audio enregistré avec succès : recording\audio_2023-07-06_10-58-24.wav
recording\audio_2023-07-06_10-58-24.wav
Lecture terminée.
Lecture terminée.
Lecture terminée.
Lecture terminée.
Lect